Adicionar o path de Dados do Google Drive

In [1]:
import plotly.express as px
import pandas as pd
import gspread
import glob

Carregar os Dados da Planilha online Google Sheets

In [2]:
class GoogleSheets:
    def __init__(self):
        self.key = glob.glob("key/*.json")[0]
        self.CODE = '1AYpQ_uHm7aZ-VZwibuscpQxgJ-vXirm4PWrJDECYr1c'
        self.gc = gspread.service_account(filename=self.key)
        self.sh = self.gc.open_by_key(self.CODE)

    def visualizar_dados(self):
        ws = self.sh.worksheet('CONVIDADOS DO CASAMENTO')
        dados = ws.get_all_records()
        planilha = pd.DataFrame(dados)
        return planilha

# Create an instance of the GoogleSheets class
gs = GoogleSheets()

# Call the visualizar_dados method
CONVIDADOS_DF = pd.DataFrame(gs.visualizar_dados())

# Transformar a coluna de parentesco para minuscula.
Parentesco  = CONVIDADOS_DF["Parentesco"].str.lower()

CONVIDADOS_DF

,Nome,PARENTESCO:,Parentesco,Padrinho,Localidade:,Criança,Convite Enviado ?,Codigo,Pessoas da Fazenda
0,Elenice Inês Dos Santos,MAE DO NOIVO,Mãe do noivo,,,Não,Sim,64ddc,12
1,Luiz Felipe Linhares,,Amigo do noivo,,,Não,Sim,6606c,
2,Verônica,,Mulher Felipe Linhares,,,Não,Sim,673c1,
3,Ana Luiza Dos Santos Moreira,IRMA DO NOIVO,Madrinha do noivo,Sim,,Não,Sim,6894f,
4,Lucas Junior Dos Santos Moreira,IRMAO DO NOIVO,Irmão do noivo,,,Não,Sim,69eef,
...,...,...,...,...,...,...,...,...,...
153,Marcia Lopes Coelho,,Mulher do pastor que vai realizar o casamento,,,Não,Sim,8841f,
154,Pastor Cláudio,,O pastor que fazer o casamento,,,Não,Sim,f3cf5,
155,Luiz paulo,,Amigo do noivo,,,Não,Sim,f899,
156,Vanessa Gomes,,Amigo do noivo,,,Não,Sim,2359y,


Verificar os nomes dos convites não enviados.

In [3]:
CONVIDADOS_DF[CONVIDADOS_DF["Convite Enviado ?"] == "Não"]

,Nome,PARENTESCO:,Parentesco,Padrinho,Localidade:,Criança,Convite Enviado ?,Codigo,Pessoas da Fazenda
10,Andréia de Fátima Santos,,Prima do noivo,,fazenda,Não,Não,71781,
11,Carlos André dos Santos,PRIMO DO NOIVO,Primo do noivo,,fazenda,Não,Não,727fd,
21,Célia Martins dos Santos,,Tia do Noivo,,fazenda,Não,Não,7d39a,
25,Heloísa Martins dos Santos,,Prima do noivo,,fazenda,Não,Não,8142a,
26,Daniel,,Marido da Prima do Noivo,,fazenda,Não,Não,82229,
27,Bianca Martins,,Sobrinha do Noivo,,fazenda,Não,Não,8302a,
28,Raquel Martins,,Prima do Noivo,,fazenda,Não,Não,83df2,
29,Petruquio,,Primo do Noivo,,fazenda,Não,Não,84c02,
31,Mateus Gomes,PRIMO DO NOIVO,Primo do noivo,,,Não,Não,86a71,
141,Viviane Fernandes dos Santos,,prima do noivo,,fazenda,Não,Não,e92a8,


Criar vizualização para ver os convites enviados

In [4]:
# Gerando o conjunto análisado.
convites_enviados = CONVIDADOS_DF['Convite Enviado ?'].value_counts().reset_index()
convites_enviados.columns = ["Convite Enviado ?", "Quantidade"]


convites_enviados

# prompt: gere um gráfico de rosca com plotly com a quantidade de convites enviados
fig = px.pie(convites_enviados, values='Quantidade', names='Convite Enviado ?', title='Convites Enviados', hole=0.6)
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Gerar os Convites

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import shutil
import os
import pandas as pd
import uuid


# Classe do Sistema
class Sistema:
    @staticmethod
    def apagar_conteudo_diretorio(diretorio):
        if os.path.exists(diretorio):
            for item in os.listdir(diretorio):
                item_path = os.path.join(diretorio, item)
                try:
                    if os.path.isfile(item_path) or os.path.islink(item_path):
                        os.unlink(item_path)  # Remove arquivos ou links simbólicos
                    elif os.path.isdir(item_path):
                        shutil.rmtree(item_path)  # Remove diretórios recursivamente
                except Exception as e:
                    print(f'Erro ao apagar {item_path}: {e}')
            print(f'Todo o conteúdo do diretório {diretorio} foi apagado com sucesso.')
        else:
            print(f'O diretório {diretorio} não existe.')

# Exemplo de uso
diretorio_para_limpar = '/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados'
Sistema.apagar_conteudo_diretorio(diretorio_para_limpar)

# Definir fontes
font1 = ImageFont.truetype("/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/font/font.otf", size=45)
font2 = ImageFont.truetype("/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/font/font.otf", size=30)
path_imagem_base = "/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/template/template.png"
path_imagem_output = "/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados"

def gerar_convite(convidado, codigo,path_imagem_base,path_imagem_output,font1,font2):

    imagem = Image.open(path_imagem_base).convert("RGBA")

    lapis = ImageDraw.Draw(imagem)

    dados_imagem = imagem.size
    dados_texto = len(convidado)

    print(f"Pessoa: {convidado}, Tamanho do texto: {dados_texto}")

    # Calcular a posição x para centralizar o texto
    # Use getbbox instead of getsize
    posicao_x = (dados_imagem[0] - font1.getbbox(convidado)[2]) // 2

    # Nome do Convidado
    lapis.text((posicao_x, 1170), text=convidado, fill="#000", font=font1) # Update to use posicao_x

    # Código do Convite
    lapis.text((165, 1265), text=f"código: {codigo}", fill="#000", font=font2)

    # Salvar a imagem com o nome do convidado.
    imagem.save(f"{path_imagem_output}/CONVITE - {convidado}.png")

for convidado in CONVIDADOS_DF.values:
    gerar_convite(convidado[0], convidado[7],path_imagem_base,path_imagem_output,font1,font2)

#lista_df.to_excel("/content/drive/MyDrive/Colab Notebooks/GeradorConvitesCasamento/convites_gerados/LISTA.xlsx", index=False)